***libraries and variables***



In [ ]:
# @title
from google.colab import drive
drive.mount('/content/OrangeOpen')
#------------------------------------------->
import pandas as pd
from datetime import datetime
from datetime import timedelta
#------------------------------------------->
from google.colab import auth
auth.authenticate_user()
Id_Project='orangedatabase'
Id_dataset='orangedatabase.Orange_Open'
from google.cloud import bigquery
bq = bigquery.Client(project=Id_Project)
BDD_Orange = bq.dataset(dataset_id=Id_dataset)
job_config = bigquery.LoadJobConfig(autodetect= True)

Mounted at /content/OrangeOpen


***Recuperation des bases de données***

In [ ]:
# @title
df_Date_Table=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Date_Table.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Effectif_Global=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/Parametres/Effectif_Global.csv' , delimiter=";" , decimal=",", encoding='unicode_escape')
df_Base_Commerce=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Commerce.csv' , delimiter=";" , decimal=",")
df_Base_RT=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_RT.csv' , delimiter=";" , decimal=",")
df_Base_CmdFibre=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Commande_Fibre.csv' , delimiter=";" , decimal=",")


***Commerce Global***

In [ ]:
# @title
Id_table_Commerce='orangedatabase.Orange_Open.Tab_Commerce'
Id_table_Commerce_Ext='orangedatabase.Orange_Open.Tab_Commerce_Ext'

df_Effectif = df_Effectif_Global.drop(columns=['Jour', 'Mois','Semaine','Annee','Semaine_Format_2','Trimestre','Datation'])
df_1 = df_Base_Commerce.drop(columns=['Lib_Equipe', 'KPI','SousKpi','KPI_Item','Datation'])
df_2 = df_Base_RT.drop(columns=['Lib_Equipe', 'FluxArrivee','Offre', 'Univers', 'MetierTraitant','exploitables', 'Reit1J', 'Reit7J', 'Appels_1min', 'NbreTransfertsAccompAveug','Datation'])
df_3 = df_Base_Commerce.drop(columns=['Lib_Equipe', 'FamilleKpiHisto','SousKpi', 'KPI','Datation'])
df_4 = df_Base_CmdFibre.drop(columns=['Lib_Equipe','Datation'])
df_Ext = df_Base_Commerce.drop(columns=['Lib_Equipe','SousKpi', 'KPI','Datation'])


df_1['FamilleKpiHisto'] = df_1['FamilleKpiHisto'].apply(lambda x: x.replace(' ', '_'))
df_1['FamilleKpiHisto'] = df_1['FamilleKpiHisto'].apply(lambda x: x.replace('(DONT_DOM)', 'DOM'))

pivoted=pd.pivot_table(df_1, values='Total', index =['Date_Format_1','CUID','Flux'] ,columns=['FamilleKpiHisto'], aggfunc='sum')
df_1=pd.DataFrame(pivoted.to_records())

pivoted=pd.pivot_table(df_2, values='NbreAppelTraite', index =['Date_Format_1','CUID','Flux'] ,columns=['Sous_Flux'], aggfunc='sum')
df_2=pd.DataFrame(pivoted.to_records())


pivoted=pd.pivot_table(df_3, values='Total', index =['Date_Format_1','CUID','Flux'] ,columns=['KPI_Item'], aggfunc='sum')
df_3=pd.DataFrame(pivoted.to_records())

dfg = pd.concat([df_1, df_2, df_3, df_4], join='outer')
t_table = pd.pivot_table(dfg, index=['Date_Format_1','CUID', 'Flux'],aggfunc = 'sum')
dfg=pd.DataFrame(t_table.to_records())

#display(dfg)

dfg = pd.merge(left=dfg, right=df_Date_Table, how='left' , on='Date_Format_1')
dfg = pd.merge(left=dfg, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])
df_Ext = pd.merge(left=df_Ext, right=df_Date_Table, how='left' , on='Date_Format_1')
df_Ext = pd.merge(left=df_Ext, right=df_Effectif, how='left' , on=['Date_Format_1','CUID'])

dfg.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Commerce.csv' , index=False, sep = ';' , decimal=",")

dfg['Datation']=dfg['Datation'].values.astype(str)
dfg['Datation'] = dfg['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

df_Ext['Datation']=df_Ext['Datation'].values.astype(str)
df_Ext['Datation'] = df_Ext['Datation'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

table = bigquery.Table(Id_table_Commerce)
bq.delete_table(Id_table_Commerce, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(dfg, Id_table_Commerce, job_config=job_config)

table = bigquery.Table(Id_table_Commerce_Ext)
bq.delete_table(Id_table_Commerce_Ext, not_found_ok=True)
table = bq.create_table(table)
job = bq.load_table_from_dataframe(df_Ext, Id_table_Commerce_Ext, job_config=job_config)


In [ ]:
# @title
df_Import_AppelCourts2=pd.read_excel('OrangeOpen/My Drive/Project Orange Open/Imports/Appels Courts.xlsx' , sheet_name='Data_appels_Courts_Evita_O-M'  , skiprows=24 , usecols=[11, 12, 13 , 14 , 15, 16, 17, 18, 19] , names=['Datation', 'Demi-heure', 'Competence','Offre', 'Type_Origine', 'Presentes', 'Traites', 'Appels_Courts', 'Appels_Pris_En_Charge'])




df_Import_AppelCourts2.dropna(subset=['Datation'], inplace=True)
df_Import_AppelCourts2 = df_Import_AppelCourts2[df_Import_AppelCourts2['Datation'].notna()]
df_Import_AppelCourts2["Datation"] = pd.to_datetime(df_Import_AppelCourts2["Datation"])
df_Import_AppelCourts2['Date_Format_1'] =df_Import_AppelCourts2['Datation'].dt.strftime('%Y%m%d').astype(int)
df_Import_AppelCourts2['Datation'] = df_Import_AppelCourts2['Datation'].dt.strftime('%d/%m/%Y')



data_Competence = [['cConvAC' , 'Open'] , ['DcConvAC' , 'Open'] , ['cConvEli' , 'Open'] , ['cPaiement' , 'Open'] , ['hHlboutique' , 'HLB'] , ['hHDCtcDco' , 'Autre'] , ['hHDCtcHd' , 'Autre'] , ['hHDAdsl' , 'Autre'] , ['hHDFibre' , 'Autre'] , ['hHDRaDcoAdsl' , 'Autre'] , ['hHDRd3Adsl' , 'Autre'] , ['hHDRaDcoFibre' , 'Autre'] , ['hHDRd3Fibre' , 'Autre'] , ['DmobileAC' , 'Open'] , ['mobileAC' , 'Open'] , ['hHDTvZne' , 'Autre'] , ['cConvDart' , 'DART'] , ['mHnoAC' , 'Autre'] , ['hHDHDeskOAsst' , 'Help Desk'] , ['hHDDom' , 'Autre'] , ['hHDNuit' , 'Autre'] , ['hHDFibreDom' , 'Autre'] , ['hHDRd3Dom' , 'Autre'] , ['hHDHomelan' , 'Autre'] , ['HLDPAPFibGP' , 'HLD'] , ['OMFSerClt' , 'Orange Money'] , ['HLDACFull' , 'HLD'] , ['cSpaAppui' , 'SPA'] , ['HFAPPUISPA' , 'SPA'] , ['hHDAutonomie' , 'Autonomie'] , ['mMobAC' , 'Open'] , ['mMobEli' , 'Open']]
df_data_Competence = pd.DataFrame(data_Competence, columns=['Competence', 'Flux'])
df_Import_AppelCourts2 = pd.merge(df_Import_AppelCourts2, df_data_Competence,  on ='Competence',  how ='left')

df_Import_AppelCourts2.loc[(df_Import_AppelCourts2['Flux'] == 'Open') &  (df_Import_AppelCourts2['Offre'] == 'ORANGE'), 'Sous_Flux'] = 'Mobile'
df_Import_AppelCourts2.loc[(df_Import_AppelCourts2['Flux'] == 'Open') &  (df_Import_AppelCourts2['Offre'] == 'ORANGE OPEN'), 'Sous_Flux'] = 'ADSL'
df_Import_AppelCourts2.loc[(df_Import_AppelCourts2['Flux'] == 'Open') &  (df_Import_AppelCourts2['Offre'] == 'ORANGE OPEN FIBRE'), 'Sous_Flux'] = 'Fibre'
df_Import_AppelCourts2.loc[(df_Import_AppelCourts2['Flux'] == 'Open') &  (df_Import_AppelCourts2['Offre'] == 'NC'), 'Sous_Flux'] = 'Premium'


display(df_Import_AppelCourts2)








# @title
df_Import_FluxQuotidien=pd.read_excel('OrangeOpen/My Drive/Project Orange Open/Imports/Flux Quotidien.xlsx' , sheet_name='Data_Qs_Evita'  , skiprows=34 , usecols=[1, 2, 3 , 4 , 5, 6, 7,8,9,10,11,12,13,14,15,16,17,18,19,20,21] , names=['Datation', 'Univers', 'Type_Origine','Tendance', 'Type_Activite', 'Competence','Offre', 'Demi-heure', 'Presentes', 'Presentes_nominal', 'Presentes_en_debo', 'Traites', 'Traites_nominal', 'Traites_en_debo', 'Dissuades', 'Abandons', 'Abandons_courts', 'Abandons_longs', 'DMSonnerie', 'DMR', 'DMAbd'])
#df_Import_FluxQuotidien=pd.read_excel('OrangeOpen/My Drive/Project Orange Open/Imports/FluxQuotidien.xlsx' , sheet_name='FluxQuotidien' , usecols=[1, 2, 3 , 4 , 5, 6, 7,8,9,10,11,12,13,14,15,16,17,18,19,20,21] , names=['Datation', 'Type_Origine', 'Univers', 'Competence','Tendance', 'Type_Activite','Offre', 'Demi-heure', 'Presentes', 'Presentes_nominal', 'Presentes_en_debo', 'Traites', 'Traites_nominal', 'Traites_en_debo', 'Dissuades', 'Abandons', 'Abandons_courts', 'Abandons_longs', 'DMSonnerie', 'DMR', 'DMAbd'])
df_Base_FluxQuotidien=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_FluxQuotidien.csv' , delimiter=";" , decimal=",")

df_Import_FluxQuotidien["Datation"] = pd.to_datetime(df_Import_FluxQuotidien["Datation"])
df_Import_FluxQuotidien['Date_Format_1'] =df_Import_FluxQuotidien['Datation'].dt.strftime('%Y%m%d').astype(int)
df_Import_FluxQuotidien['Datation'] = df_Import_FluxQuotidien['Datation'].dt.strftime('%d/%m/%Y')

data_Competence = [['cConvAC' , 'Open'] , ['DcConvAC' , 'Open'] , ['cConvEli' , 'Open'] , ['cPaiement' , 'Open'] , ['hHlboutique' , 'HLB'] , ['hHDCtcDco' , 'Autre'] , ['hHDCtcHd' , 'Autre'] , ['hHDAdsl' , 'Autre'] , ['hHDFibre' , 'Autre'] , ['hHDRaDcoAdsl' , 'Autre'] , ['hHDRd3Adsl' , 'Autre'] , ['hHDRaDcoFibre' , 'Autre'] , ['hHDRd3Fibre' , 'Autre'] , ['DmobileAC' , 'Open'] , ['mobileAC' , 'Open'] , ['hHDTvZne' , 'Autre'] , ['cConvDart' , 'DART'] , ['mHnoAC' , 'Autre'] , ['hHDHDeskOAsst' , 'Help Desk'] , ['hHDDom' , 'Autre'] , ['hHDNuit' , 'Autre'] , ['hHDFibreDom' , 'Autre'] , ['hHDRd3Dom' , 'Autre'] , ['hHDHomelan' , 'Autre'] , ['HLDPAPFibGP' , 'HLD'] , ['OMFSerClt' , 'Orange Money'] , ['HLDACFull' , 'HLD'] , ['cSpaAppui' , 'SPA'] , ['HFAPPUISPA' , 'SPA'] , ['hHDAutonomie' , 'Autonomie'] , ['mMobAC' , 'Open'] , ['mMobEli' , 'Open']]
df_data_Competence = pd.DataFrame(data_Competence, columns=['Competence', 'Flux'])
df_Import_FluxQuotidien = pd.merge(df_Import_FluxQuotidien, df_data_Competence,  on ='Competence',  how ='left')

df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Open') &  (df_Import_FluxQuotidien['Offre'] == 'ORANGE'), 'Sous_Flux'] = 'Mobile'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Open') &  (df_Import_FluxQuotidien['Offre'] == 'ORANGE OPEN'), 'Sous_Flux'] = 'ADSL'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Open') &  (df_Import_FluxQuotidien['Offre'] == 'ORANGE OPEN FIBRE'), 'Sous_Flux'] = 'Fibre'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Open') &  (df_Import_FluxQuotidien['Offre'] == 'NC'), 'Sous_Flux'] = 'Premium'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'DART') &  (df_Import_FluxQuotidien['Tendance'] == 'FORFAITS'), 'Sous_Flux'] = 'Mobile'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'DART') &  (df_Import_FluxQuotidien['Tendance'] == 'CONVERGENT'), 'Sous_Flux'] = 'Convergent'

df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'HLB') &  (df_Import_FluxQuotidien['Type_Origine'] == 'Boutique Franchisée'), 'Sous_Flux'] = 'Boutique Franchisée'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'HLB') &  (df_Import_FluxQuotidien['Type_Origine'] == 'Boutique Orange'), 'Sous_Flux'] = 'Boutique Orange'
df_Import_FluxQuotidien.loc[df_Import_FluxQuotidien['Flux'] == 'Help Desk', 'Sous_Flux'] = 'Help Desk'
df_Import_FluxQuotidien.loc[df_Import_FluxQuotidien['Flux'] == 'Autonomie', 'Sous_Flux'] = 'Autonomie'
df_Import_FluxQuotidien.loc[df_Import_FluxQuotidien['Flux'] == 'Autre', 'Sous_Flux'] = 'Autre'
df_Import_FluxQuotidien.loc[df_Import_FluxQuotidien['Flux'] == 'SPA', 'Sous_Flux'] = 'SPA'

df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'HLD') &  (df_Import_FluxQuotidien['Competence'] == 'HLDPAPFibGP'), 'Sous_Flux'] = 'PAP'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'HLD') &  (df_Import_FluxQuotidien['Type_Origine'] == 'Boutique Franchisée'), 'Sous_Flux'] = 'Boutique Franchisée'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'HLD') &  (df_Import_FluxQuotidien['Type_Origine'] == 'Boutique Orange'), 'Sous_Flux'] = 'Boutique Orange'

df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Orange Money') &  (df_Import_FluxQuotidien['Type_Origine'] == 'Revendeur'), 'Sous_Flux'] = 'Revendeur'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Orange Money') &  (df_Import_FluxQuotidien['Type_Origine'] == 'Boutique Orange'), 'Sous_Flux'] = 'Boutique Orange'
df_Import_FluxQuotidien.loc[(df_Import_FluxQuotidien['Flux'] == 'Orange Money') &  (df_Import_FluxQuotidien['Type_Origine'] == 'SVI'), 'Sous_Flux'] = 'SVI'

#df_Import_FluxQuotidien.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_FluxQuotidien.csv' , index=False, sep = ';' , decimal=",")

if df_Import_FluxQuotidien['Flux'].isna().sum() >0 or df_Import_FluxQuotidien['Sous_Flux'].isna().sum() >0:
 raise Exception("Une ou plusieurs affectation manquantes")

df_Import_FluxQuotidien['Duree_Reponse'] = df_Import_FluxQuotidien['DMR'] * df_Import_FluxQuotidien['Traites']

df_Import_FluxQuotidien_Dates=df_Import_FluxQuotidien["Date_Format_1"].unique()
print(df_Import_FluxQuotidien_Dates)
df_Base_FluxQuotidien = df_Base_FluxQuotidien[~df_Base_FluxQuotidien['Date_Format_1'].isin(df_Import_FluxQuotidien_Dates)]
df_Base_FluxQuotidien = pd.concat([df_Base_FluxQuotidien,df_Import_FluxQuotidien], keys=["Date_Format_1", "Date_Format_1"])

df_Import_FluxQuotidien.to_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_FluxQuotidien.csv' , index=False, sep = ';' , decimal=",")



































# @title
df_Import_AppelCourts=pd.read_excel('OrangeOpen/My Drive/Project Orange Open/Imports/Appels Courts.xlsx' , sheet_name='Data_appels_Courts_Evita_O-M'  , skiprows=24 , usecols=[1, 2, 3 , 4 , 5, 6, 7,8] , names=['Datation', 'Demi-heure', 'CUID','Tendance','Offre', 'Type_Activite', 'Type_Origine', 'Appels_Courts'])
df_Base_AppelCourts=pd.read_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_AppelCourts.csv' , delimiter=";" , decimal=",")

df_Import_AppelCourts.dropna(subset=['Datation'], inplace=True)
df_Import_AppelCourts["Datation"] = pd.to_datetime(df_Import_AppelCourts["Datation"])
df_Import_AppelCourts['Date_Format_1'] =df_Import_AppelCourts['Datation'].dt.strftime('%Y%m%d').astype(int)
df_Import_AppelCourts['Datation'] = df_Import_AppelCourts['Datation'].dt.strftime('%d/%m/%Y')

df_Import_AppelCourts.to_csv('OrangeOpen/My Drive/Project Orange Open/EXPORT/Tab_Open_Data_AppelCourts.csv' , index=False, sep = ';' , decimal=",")

data_Type_Activite = [['Hot Line Boutique' , 'HLB'] ,['Info_Facture' , 'DART'] ,['Help Desk Orange Assistance' , 'Help Desk'] ,['Orange Money France' , 'Orange Money'] ,['Soutien plateau Assistance' , 'SPA'] ,['Porte à Porte' , 'HLD'] ,['Reprise Actes HLD' , 'HLD'] ,['Fidélisation Hld' , 'HLD'] ,['Information Offre et Process' , 'HLD'] ,['Service Client HLD' , 'HLD'] ,['Libération IMEI' , 'HLD']]
df_data_Type_Activite = pd.DataFrame(data_Type_Activite, columns=['Type_Activite', 'Flux'])
df_Import_AppelCourts = pd.merge(df_Import_AppelCourts, df_data_Type_Activite,  on ='Type_Activite',  how ='left')

df_Import_AppelCourts.loc[df_Import_AppelCourts['Offre'] == 'MIXTE' , 'Flux'] = 'Autre'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Tendance'] == 'HD') &  (df_Import_AppelCourts['Flux'].isnull()), 'Flux'] = 'Autre'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Tendance'] == 'NC') &  (df_Import_AppelCourts['Flux'].isnull()), 'Flux'] = 'Autre'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Tendance'] == 'THD') &  (df_Import_AppelCourts['Flux'].isnull()), 'Flux'] = 'Autre'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Tendance'] == 'CONVERGENT') &  (df_Import_AppelCourts['Flux'].isnull()), 'Flux'] = 'Open'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Tendance'] == 'FORFAITS') &  (df_Import_AppelCourts['Flux'].isnull()), 'Flux'] = 'Open'
df_Import_AppelCourts.loc[df_Import_AppelCourts['Flux'] == 'Autre' , 'Sous_Flux'] = 'Autre'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Open') &  (df_Import_AppelCourts['Offre'] == 'ORANGE'), 'Sous_Flux'] = 'Mobile'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Open') &  (df_Import_AppelCourts['Offre'] == 'ORANGE OPEN'), 'Sous_Flux'] = 'ADSL'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Open') &  (df_Import_AppelCourts['Offre'] == 'ORANGE OPEN FIBRE'), 'Sous_Flux'] = 'Fibre'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Open') &  (df_Import_AppelCourts['Offre'] == 'NC'), 'Sous_Flux'] = 'Premium'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'DART') &  (df_Import_AppelCourts['Tendance'] == 'FORFAITS'), 'Sous_Flux'] = 'Mobile'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'DART') &  (df_Import_AppelCourts['Tendance'] == 'CONVERGENT'), 'Sous_Flux'] = 'Convergent'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'HLB') &  (df_Import_AppelCourts['Type_Origine'] == 'Boutique Franchisée'), 'Sous_Flux'] = 'Boutique Franchisée'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'HLB') &  (df_Import_AppelCourts['Type_Origine'] == 'Boutique Orange'), 'Sous_Flux'] = 'Boutique Orange'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Orange Money') &  (df_Import_AppelCourts['Type_Origine'] == 'Revendeur'), 'Sous_Flux'] = 'Revendeur'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Orange Money') &  (df_Import_AppelCourts['Type_Origine'] == 'Boutique Orange'), 'Sous_Flux'] = 'Boutique Orange'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'Orange Money') &  (df_Import_AppelCourts['Type_Origine'] == 'SVI'), 'Sous_Flux'] = 'SVI'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'HLD') &  (df_Import_AppelCourts['Type_Origine'] == 'Boutique Franchisée'), 'Sous_Flux'] = 'Boutique Franchisée'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'HLD') &  (df_Import_AppelCourts['Type_Origine'] == 'Boutique Orange'), 'Sous_Flux'] = 'Boutique Orange'
df_Import_AppelCourts.loc[(df_Import_AppelCourts['Flux'] == 'HLD') &  (df_Import_AppelCourts['Type_Activite'] == 'Porte à Porte'), 'Sous_Flux'] = 'PAP'
df_Import_AppelCourts.loc[df_Import_AppelCourts['Flux'] == 'Help Desk', 'Sous_Flux'] = 'Help Desk'
df_Import_AppelCourts.loc[df_Import_AppelCourts['Flux'] == 'Autonomie', 'Sous_Flux'] = 'Autonomie'
df_Import_AppelCourts.loc[df_Import_AppelCourts['Flux'] == 'SPA', 'Sous_Flux'] = 'SPA'



if df_Import_AppelCourts['Flux'].isna().sum() >0 or df_Import_AppelCourts['Sous_Flux'].isna().sum() >0:
 raise Exception("Une ou plusieurs affectation manquantes")

df_Import_AppelCourts_Dates=df_Import_AppelCourts["Date_Format_1"].unique()
print(df_Import_AppelCourts_Dates)
df_Base_AppelCourts = df_Base_AppelCourts[~df_Base_AppelCourts['Date_Format_1'].isin(df_Import_AppelCourts_Dates)]
df_Base_AppelCourts = pd.concat([df_Base_AppelCourts,df_Import_AppelCourts], keys=["Date_Format_1", "Date_Format_1"])

df_Base_AppelCourts.to_csv('OrangeOpen/My Drive/Project Orange Open/BDD/Tab_Open_Data_AppelCourts.csv' , index=False, sep = ';' , decimal=",")






